# Data Preprocessing for Ames Iowa Housing Dataset

## Goals
- Clean missing values
- Encode categorical variables
- Transform quantitative variables

## Imports and Data Loading

In [51]:
import pandas as pd
from scipy.stats import skew, boxcox

In [52]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [53]:
train_numeric_features = train.select_dtypes(include=['float64', 'int64']).columns.difference(['Id', 'MSSubClass'])

In [54]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [55]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Fill Missing Values

This will initially be done just by using mean/median, 0, or None depending on which is more appropriate

In [56]:
def preprocess_data(train, test):
    # Copy the train and test DataFrames to avoid changing the original data
    train_processed = train.copy().drop(columns=['Id'])
    test_processed = test.copy().drop(columns=['Id'])

    train_processed['MSSubClass'] = train_processed['MSSubClass'].astype(str)
    test_processed['MSSubClass'] = test_processed['MSSubClass'].astype(str)

    # Fill missing values for variables in train
    train_processed['Electrical'] = train_processed['Electrical'].fillna(train_processed['Electrical'].mode()[0])
    train_processed['Alley'] = train_processed['Alley'].fillna('None')
    train_processed['MiscFeature'] = train_processed['MiscFeature'].fillna('None')
    train_processed['Fence'] = train_processed['Fence'].fillna('None')
    train_processed['LotFrontage'] = train_processed['LotFrontage'].fillna(0)
    train_processed['GarageYrBlt'] = train_processed['GarageYrBlt'].fillna(0)
    
    # Apply custom transformations based on conditions for train data
    train_processed['PoolQC'] = train_processed.apply(lambda row: validity_check(row, 'PoolArea', 'PoolQC', train_processed), axis=1)
    train_processed['FireplaceQu'] = train_processed.apply(lambda row: validity_check(row, 'Fireplaces', 'FireplaceQu', train_processed), axis=1)
    train_processed['MasVnrArea'] = train_processed['MasVnrArea'].fillna(0)
    train_processed['MasVnrType'] = train_processed.apply(lambda row: masonry_check(row, train_processed), axis=1)
    
    # Apply validity check for basement columns in train data
    train_processed['BsmtCond'] = train_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtCond', train_processed), axis=1)
    train_processed['BsmtQual'] = train_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtQual', train_processed), axis=1)
    train_processed['BsmtExposure'] = train_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtExposure', train_processed), axis=1)
    train_processed['BsmtFinType1'] = train_processed.apply(lambda row: validity_check(row, 'BsmtFinSF1', 'BsmtFinType1', train_processed), axis=1)
    train_processed['BsmtFinType2'] = train_processed.apply(lambda row: validity_check(row, 'BsmtFinSF1', 'BsmtFinType2', train_processed), axis=1)

    # Apply validity check for garage columns in train data
    train_processed['GarageQual'] = train_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageQual', train_processed), axis=1)
    train_processed['GarageCond'] = train_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageCond', train_processed), axis=1)
    train_processed['GarageFinish'] = train_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageFinish', train_processed), axis=1)
    train_processed['GarageType'] = train_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageType', train_processed), axis=1)

    # Now process the test data using the same transformations
    test_processed['Electrical'] = test_processed['Electrical'].fillna(train_processed['Electrical'].mode()[0])  # Use train mode
    test_processed['Alley'] = test_processed['Alley'].fillna('None')
    test_processed['MiscFeature'] = test_processed['MiscFeature'].fillna('None')
    test_processed['Fence'] = test_processed['Fence'].fillna('None')
    test_processed['LotFrontage'] = test_processed['LotFrontage'].fillna(0)
    test_processed['GarageYrBlt'] = test_processed['GarageYrBlt'].fillna(0)

    #Additional NaN values present in test
    test_processed['GarageArea'] = test_processed['GarageArea'].fillna(0)
    test_processed['GarageCars'] = test_processed['GarageCars'].fillna(0)
    test_processed['BsmtFinSF1'] = test_processed['BsmtFinSF1'].fillna(0)
    test_processed['BsmtFinSF2'] = test_processed['BsmtFinSF2'].fillna(0)
    test_processed['BsmtUnfSF'] = test_processed['BsmtUnfSF'].fillna(0)
    test_processed['TotalBsmtSF'] = test_processed['TotalBsmtSF'].fillna(0)
    test_processed['BsmtFullBath'] = test_processed['BsmtFullBath'].fillna(0)
    test_processed['BsmtHalfBath'] = test_processed['BsmtHalfBath'].fillna(0)
    test_processed['Exterior1st'] = test_processed['Exterior1st'].fillna(train_processed['Exterior1st'].mode()[0])
    test_processed['Exterior2nd'] = test_processed['Exterior2nd'].fillna(train_processed['Exterior2nd'].mode()[0])
    test_processed['KitchenQual'] = test_processed['KitchenQual'].fillna(train_processed['KitchenQual'].mode()[0])
    test_processed['SaleType'] = test_processed['SaleType'].fillna(train_processed['SaleType'].mode()[0])
    test_processed['Utilities'] = test_processed['Utilities'].fillna(train_processed['Utilities'].mode()[0])
    test_processed['Functional'] = test_processed['Functional'].fillna(train_processed['Functional'].mode()[0])
    test_processed['MSZoning'] = test_processed['MSZoning'].fillna(train_processed['MSZoning'].mode()[0])

    # Apply custom transformations to the test data using train values for consistency
    test_processed['PoolQC'] = test_processed.apply(lambda row: validity_check(row, 'PoolArea', 'PoolQC', train_processed), axis=1)
    test_processed['FireplaceQu'] = test_processed.apply(lambda row: validity_check(row, 'Fireplaces', 'FireplaceQu', train_processed), axis=1)
    test_processed['MasVnrArea'] = test_processed['MasVnrArea'].fillna(0)
    test_processed['MasVnrType'] = test_processed.apply(lambda row: masonry_check(row, train_processed), axis=1)
    
    # Apply validity check for basement columns in test data
    test_processed['BsmtCond'] = test_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtCond', train_processed), axis=1)
    test_processed['BsmtQual'] = test_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtQual', train_processed), axis=1)
    test_processed['BsmtExposure'] = test_processed.apply(lambda row: validity_check(row, 'TotalBsmtSF', 'BsmtExposure', train_processed), axis=1)
    test_processed['BsmtFinType1'] = test_processed.apply(lambda row: validity_check(row, 'BsmtFinSF1', 'BsmtFinType1', train_processed), axis=1)
    test_processed['BsmtFinType2'] = test_processed.apply(lambda row: validity_check(row, 'BsmtFinSF1', 'BsmtFinType2', train_processed), axis=1)

    # Apply validity check for garage columns in test data
    test_processed['GarageQual'] = test_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageQual', train_processed), axis=1)
    test_processed['GarageCond'] = test_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageCond', train_processed), axis=1)
    test_processed['GarageFinish'] = test_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageFinish', train_processed), axis=1)
    test_processed['GarageType'] = test_processed.apply(lambda row: validity_check(row, 'GarageArea', 'GarageType', train_processed), axis=1)

    return train_processed, test_processed

def validity_check(row, check_col, target_col, train, median=False):
    """
    If check_col != 0 and target_col is missing, fill with median of target_col.
    If check_col == 0 and target_col is missing, fill with 'None'.
    Otherwise, keep original value.
    """
    if pd.isna(row[target_col]):
        if row[check_col] != 0:
            if median:
                return train[target_col].median()
            else:
                return train[target_col].mode()[0]
        else:
            return 'None'
    return row[target_col] 

def masonry_check(row, train):
    if pd.isna(row['MasVnrType']):
        if row['MasVnrArea'] <= 10:
            return 'None'
        else:
            return train['MasVnrType'].mode()[0]
    return row['MasVnrType'] 

train_processed, test_processed = preprocess_data(train, test)

In [57]:
train_processed.isna().any().sum(), test_processed.isna().any().sum()

(0, 0)

## Feature Engineering
- *isRemodel* - determine if house has been remodelled
- *yearsBetweenBuildRemodel* - determine how many years it has been since remodelling

In [58]:
def feature_engineering(train, test):
    # Apply feature engineering to the train dataset
    train['hasRemodel'] = train.apply(lambda row: 1 if row['YearBuilt'] != row['YearRemodAdd'] else 0, axis=1)
    train['yearsBetweenBuildRemodel'] = train['YearRemodAdd'] - train['YearBuilt']
    train['twoExteriors'] = train.apply(lambda row: 1 if row['Exterior1st'] != row['Exterior2nd'] else 0, axis=1)
    train['bathroomCount'] = train['BsmtFullBath'] + train['FullBath'] + 0.5 * (train['BsmtHalfBath'] + train['HalfBath'])
    train['totalDeckSF'] = train['WoodDeckSF'] + train['OpenPorchSF'] + train['EnclosedPorch'] + train['3SsnPorch'] + train['ScreenPorch']
    train['typesOfDeck'] = train[['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].apply(lambda x: (x > 0).sum(), axis=1)
    train['multipleDeckType'] = train['typesOfDeck'].apply(lambda x: 1 if x >= 2 else 0)
    train['totalIndoorSF'] = train['TotalBsmtSF'] + train['GrLivArea']
    train['totalHousingArea'] = train['TotalBsmtSF'] + train['GrLivArea'] + train['GarageArea'] + train['totalDeckSF']
    
    train['hasDeck'] = train['typesOfDeck'].apply(lambda x: 1 if x != 0 else 0)
    train['hasGarage'] = train['GarageArea'].apply(lambda x: 1 if x != 0 else 0)
    train['hasBsmt'] = train['TotalBsmtSF'].apply(lambda x: 1 if x != 0 else 0)
    train['has2ndFlr'] = train['2ndFlrSF'].apply(lambda x: 1 if x != 0 else 0)
    train['hasPool'] = train['PoolArea'].apply(lambda x: 1 if x != 0 else 0)
    train['hasFireplace'] = train['Fireplaces'].apply(lambda x: 1 if x != 0 else 0)
    train['hasLowQualSF'] = train['LowQualFinSF'].apply(lambda x: 1 if x != 0 else 0)
    
    train['2nd1stFlrSFRatio'] = train['2ndFlrSF'] / (train['1stFlrSF'] + 1e-6)
    train['1stFlrTotalSFProp'] = train['1stFlrSF'] / (train['GrLivArea'] + 1e-6)
    train['2ndFlrTotalSFProp'] = train['2ndFlrSF'] / (train['GrLivArea'] + 1e-6)
    train['IndoorLotSFRatio'] = train['totalIndoorSF'] / (train['LotArea'] + 1e-6)
    train['TotalLotSFRatio'] = train['totalHousingArea'] / (train['LotArea'] + 1e-6)
    train['FrontageLotRatio'] = train['LotFrontage'] / (train['LotArea'] + 1e-6)
    train['BsmtUpstairsRatio'] = train['TotalBsmtSF'] / (train['GrLivArea'] + 1e-6)

    # Apply the same feature engineering to the test dataset
    test['hasRemodel'] = test.apply(lambda row: 1 if row['YearBuilt'] != row['YearRemodAdd'] else 0, axis=1)
    test['yearsBetweenBuildRemodel'] = test['YearRemodAdd'] - test['YearBuilt']
    test['twoExteriors'] = test.apply(lambda row: 1 if row['Exterior1st'] != row['Exterior2nd'] else 0, axis=1)
    test['bathroomCount'] = test['BsmtFullBath'] + test['FullBath'] + 0.5 * (test['BsmtHalfBath'] + test['HalfBath'])
    test['totalDeckSF'] = test['WoodDeckSF'] + test['OpenPorchSF'] + test['EnclosedPorch'] + test['3SsnPorch'] + test['ScreenPorch']
    test['typesOfDeck'] = test[['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].apply(lambda x: (x > 0).sum(), axis=1)
    test['multipleDeckType'] = test['typesOfDeck'].apply(lambda x: 1 if x >= 2 else 0)
    test['totalIndoorSF'] = test['TotalBsmtSF'] + test['GrLivArea']
    test['totalHousingArea'] = test['TotalBsmtSF'] + test['GrLivArea'] + test['GarageArea'] + test['totalDeckSF']
    
    test['hasDeck'] = test['typesOfDeck'].apply(lambda x: 1 if x != 0 else 0)
    test['hasGarage'] = test['GarageArea'].apply(lambda x: 1 if x != 0 else 0)
    test['hasBsmt'] = test['TotalBsmtSF'].apply(lambda x: 1 if x != 0 else 0)
    test['has2ndFlr'] = test['2ndFlrSF'].apply(lambda x: 1 if x != 0 else 0)
    test['hasPool'] = test['PoolArea'].apply(lambda x: 1 if x != 0 else 0)
    test['hasFireplace'] = test['Fireplaces'].apply(lambda x: 1 if x != 0 else 0)
    test['hasLowQualSF'] = test['LowQualFinSF'].apply(lambda x: 1 if x != 0 else 0)
    
    test['2nd1stFlrSFRatio'] = test['2ndFlrSF'] / (test['1stFlrSF'] + 1e-6)
    test['1stFlrTotalSFProp'] = test['1stFlrSF'] / (test['GrLivArea'] + 1e-6)
    test['2ndFlrTotalSFProp'] = test['2ndFlrSF'] / (test['GrLivArea'] + 1e-6)
    test['IndoorLotSFRatio'] = test['totalIndoorSF'] / (test['LotArea'] + 1e-6)
    test['TotalLotSFRatio'] = test['totalHousingArea'] / (test['LotArea'] + 1e-6)
    test['FrontageLotRatio'] = test['LotFrontage'] / (test['LotArea'] + 1e-6)
    test['BsmtUpstairsRatio'] = test['TotalBsmtSF'] / (test['GrLivArea'] + 1e-6)

    return train, test

train_processed, test_processed = feature_engineering(train_processed, test_processed)

### Feature Engineering Workspace

In [59]:
tmp = train.copy()

In [60]:
## First Attempt

tmp['hasRemodel'] = tmp.apply(lambda row: 1 if row['YearBuilt'] != row['YearRemodAdd'] else 0, axis=1)
tmp['yearsBetweenBuildRemodel'] = tmp['YearRemodAdd'] - tmp['YearBuilt']
tmp['twoExteriors'] = tmp.apply(lambda row: 1 if row['Exterior1st'] != row['Exterior2nd'] else 0, axis = 1)
tmp['bathroomCount'] = tmp['BsmtFullBath'] + tmp['FullBath'] + 0.5*(tmp['BsmtHalfBath'] + tmp['HalfBath'])
tmp['totalDeckSF'] = tmp['WoodDeckSF'] + tmp['OpenPorchSF'] + tmp['EnclosedPorch'] + tmp['3SsnPorch'] + tmp['ScreenPorch']
tmp['typesOfDeck'] = tmp[['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].apply(lambda x: (x > 0).sum(), axis=1)
tmp['multipleDeckType'] = tmp['typesOfDeck'].apply(lambda x: 1 if x >= 2 else 0)
tmp['totalIndoorSF'] = tmp['TotalBsmtSF'] + tmp['GrLivArea']
tmp['totalHousingArea'] = tmp['TotalBsmtSF'] + tmp['GrLivArea'] + tmp['typesOfDeck'] + tmp['GarageArea']

tmp['hasDeck'] = tmp['typesOfDeck'].apply(lambda x: 1 if x != 0 else 0)
tmp['hasGarage'] = tmp['GarageArea'].apply(lambda x: 1 if x != 0 else 0)
tmp['hasBsmt'] = tmp['TotalBsmtSF'].apply(lambda x: 1 if x != 0 else 0)
tmp['has2ndFlr'] = tmp['2ndFlrSF'].apply(lambda x: 1 if x != 0 else 0)
tmp['hasPool'] = tmp['PoolArea'].apply(lambda x: 1 if x != 0 else 0)
tmp['hasFireplace'] = tmp['Fireplaces'].apply(lambda x: 1 if x != 0 else 0)
tmp['hasLowQualSF'] = tmp['LowQualFinSF'].apply(lambda x: 1 if x != 0 else 0)

tmp['2nd1stFlrSFRatio'] = tmp['2ndFlrSF'] / tmp['1stFlrSF']
tmp['1stFlrTotalSFProp'] = tmp['1stFlrSF'] / tmp['GrLiveArea']
tmp['2ndFlrTotalSFProp'] = tmp['2ndFlrSF'] / tmp['GrLiveArea']
tmp['IndoorLotSFRatio'] = tmp['totalIndoorSF'] / tmp['LotArea']
tmp['TotalLotSFRatio'] = tmp['totalHousingArea'] / tmp['LotArea']
tmp['FrontageLotRatio'] = tmp['LotFrontage'] / tmp['LotArea']
tmp['BsmtUpstairsRatio'] = tmp['TotalBsmtSF'] / tmp['GrLivArea']

tmp.head(10)

KeyError: 'GrLiveArea'

### Notes
- I am unsure about the way I have procesed missing values for *LotFrontage*
    - Statistical testing showed correlation between *LotArea* and *LotFrontage*
    - Statistical testing showed statisical significant difference in means for *LotFrontage* based on *Neighborhood*
    - For now I have assumed the no lot frontage means no direct access to public roads, but it is not possible to confirm or deny this withou accessing additional geographic information
    - It would be useful to test model performance with different ways of filling null values
- There were many more columns in test dataset that had missing values, no special processing was done for these and based on their type a 0 or mode was used to fill missing value

## Ordinal Encoded Features
Based on analysis of a data description file provided with the dataset, there are many categorical features that have an inherint order to them and can be appropriately label encoded. The features are as follows,
- OverallQual
- OverallCond
- ExterQual
- ExterCond
- BsmtQual (references height of basement so might not be appropriate)
- BsmtCond
- HeatingQC
- KitchenQual
- FireplaceQu (references type of fireplace so might not be appropriate)
- GarageQual
- GarageCond
- PoolQc

## Label Encoded Features
Some features are binary, these can be appropriately label encoded
- Street -> isPaved (Gravel, Paved)
- CentralAir -> isCentralAir (Yes, No)

## OHE Features
The rest of the categorical features were one hot encoded

In [ ]:
def encode_cat_features(train, test):
    qual_cond_mapping = {
        'None': 0,
        "Po": 1, 
        "Fa": 2, 
        "TA": 3, 
        "Gd": 4, 
        "Ex": 5
    }

    ordinal_features = [ 
        "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC", 
        "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond", "PoolQc"
    ]

    for col in ordinal_features:
        if col in train.columns:
            train[col] = train[col].map(qual_cond_mapping)
            test[col] = test[col].map(qual_cond_mapping)

    train['isPaved'] = train['Street'].apply(lambda x: 1 if x == 'Pave' else 0)
    train['isCentralAir'] = train['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)
    train.drop(columns=['Street', 'CentralAir'], inplace=True)

    test['isPaved'] = test['Street'].apply(lambda x: 1 if x == 'Pave' else 0)
    test['isCentralAir'] = test['CentralAir'].apply(lambda x: 1 if x == 'Y' else 0)
    test.drop(columns=['Street', 'CentralAir'], inplace=True)

    cat_features = train.select_dtypes(include=['O']).columns.difference(ordinal_features + ['isPaved', 'isCentralAir'])
    train = pd.get_dummies(train, columns=cat_features, drop_first=True).astype(int)
    test = pd.get_dummies(test, columns=cat_features, drop_first=True).astype(int)

    train, test = train.align(test, join='left', axis=1, fill_value=0)
    test.drop(inplace=True, columns=['SalePrice'])

    return train, test

In [ ]:
train_processed, test_processed = encode_cat_features(train_processed, test_processed)

In [ ]:
train_processed.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Utilities_NoSeWa,SalePrice
0,65,8450,7,5,2003,2003,196,4,3,4,...,0,0,0,0,0,0,0,1,0,208500
1,80,9600,6,8,1976,1976,0,3,3,4,...,0,0,0,0,0,0,0,1,0,181500
2,68,11250,7,5,2001,2002,162,4,3,4,...,0,0,0,0,0,0,0,1,0,223500
3,60,9550,7,5,1915,1970,0,3,3,3,...,0,0,0,0,0,0,0,1,0,140000
4,84,14260,8,5,2000,2000,350,4,3,4,...,0,0,0,0,0,0,0,1,0,250000


In [ ]:
test_processed.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleCondition_Partial,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Utilities_NoSeWa
0,80,11622,5,6,1961,1961,0,3,3,3,...,0,0,0,0,0,0,0,0,1,0
1,81,14267,6,6,1958,1958,108,3,3,3,...,0,0,0,0,0,0,0,0,1,0
2,74,13830,5,5,1997,1998,0,3,3,4,...,0,0,0,0,0,0,0,0,1,0
3,78,9978,6,6,1998,1998,20,3,3,3,...,0,0,0,0,0,0,0,0,1,0
4,43,5005,8,5,1992,1992,0,4,3,4,...,0,0,0,0,0,0,0,0,1,0


## Numerical Features

### Addressing Skew
In the EDA it was discovered that all features including target were highly skewed. For some models this greatly hinders the ability to make predictions. Features with high skew will be transformed to make them more normally distributed. SalePrice will be transformed right before modelling in order to save lambda value and invert predictions.

#### Notes
- While initially thought to affect model performance, model achieved significantly higher RMSLE without transforming data

### Garage Year Built
Some buildings do not have a garage, this gives the garage year built feature some weird properties, primarily that it's minimal value is 0 and the next minimal value is 1900. This will be transformed into an age of garage so this value makes more linear sense

In [ ]:
def process_numerical_features(train, test):
    for col in train_numeric_features.difference(['YearBuilt', 'YearRemodAdd', 'YrSold', 'GarageYrBlt', 'SalePrice']):
        if abs(skew(train[col])) > 0.5:
            train[col], _ = boxcox(train[col] + 1)
            test[col], _ = boxcox(test[col] + 1)

    return train, test

# train_processed, test_processed = process_numerical_features(train_processed, test_processed)

In [ ]:
train_processed.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Utilities_NoSeWa,SalePrice
0,65,8450,7,5,2003,2003,196,4,3,4,...,0,0,0,0,0,0,0,1,0,208500
1,80,9600,6,8,1976,1976,0,3,3,4,...,0,0,0,0,0,0,0,1,0,181500
2,68,11250,7,5,2001,2002,162,4,3,4,...,0,0,0,0,0,0,0,1,0,223500
3,60,9550,7,5,1915,1970,0,3,3,3,...,0,0,0,0,0,0,0,1,0,140000
4,84,14260,8,5,2000,2000,350,4,3,4,...,0,0,0,0,0,0,0,1,0,250000


In [ ]:
test_processed.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleCondition_Partial,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Utilities_NoSeWa
0,80,11622,5,6,1961,1961,0,3,3,3,...,0,0,0,0,0,0,0,0,1,0
1,81,14267,6,6,1958,1958,108,3,3,3,...,0,0,0,0,0,0,0,0,1,0
2,74,13830,5,5,1997,1998,0,3,3,4,...,0,0,0,0,0,0,0,0,1,0
3,78,9978,6,6,1998,1998,20,3,3,3,...,0,0,0,0,0,0,0,0,1,0
4,43,5005,8,5,1992,1992,0,4,3,4,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
columns = [col for col in train_processed.columns if col != 'SalePrice'] + ['SalePrice']
train_processed = train_processed[columns]

train_processed.to_csv('train_processed.csv', index=False)
test_processed.to_csv('test_processed.csv', index=False)